In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta,date
from constants import Study_Design
from my_functions import *

In [2]:
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
pt_features = pd.read_csv('data/pt_data/pt_features.csv',delimiter=',')

In [4]:
insom_events = pd.read_csv('data/pt_data/monthly_insom_event_count.csv')

In [5]:
insom_events.head(10)

,patid,eventdate,insom_count
0,1467,2007-11-30,2
1,1467,2008-01-31,2
2,1467,2008-02-29,2
3,1467,2008-03-31,1
4,1467,2009-05-31,1
5,1467,2009-08-31,1
6,1467,2013-01-31,1
7,3432,2004-12-31,1
8,3432,2008-10-31,1
9,3549,2005-10-31,1


In [6]:
insom_events['pat_index_date'] = insom_events['patid'].map(lambda x: pt_features[pt_features['patid']==x]['index_date'].values[0])

In [7]:
insom_events['eventdate'] = pd.to_datetime(insom_events['eventdate'])
insom_events['pat_index_date'] = pd.to_datetime(insom_events['pat_index_date'])

In [8]:
too_close_to_dx_period = timedelta(days=365)*Study_Design.years_between_exposure_measurement_and_index_date
too_distant_from_dx_period = too_close_to_dx_period + timedelta(days=365)*Study_Design.years_of_exposure_measurement

In [9]:
not_too_close_to_dx = insom_events['eventdate']<(insom_events['pat_index_date']-too_close_to_dx_period)
not_before_exposure_period = insom_events['eventdate']>(insom_events['pat_index_date']-too_distant_from_dx_period)

In [10]:
relev_insom_events = insom_events[not_too_close_to_dx & not_before_exposure_period]
relev_insom_events.sort_values(by='patid').head(10)

,patid,eventdate,insom_count,pat_index_date
7,3432,2004-12-31,1,2011-11-23
9,3549,2005-10-31,1,2011-02-09
10,3549,2006-01-31,1,2011-02-09
25,15385,1998-05-31,1,2005-02-09
28,18255,1997-11-30,1,2005-08-24
39,25420,1999-10-31,1,2008-04-03
44,27025,2001-08-31,1,2009-12-15
52,29551,2003-02-28,1,2011-01-26
55,32504,2002-10-31,1,2011-06-07
57,37099,2001-09-30,1,2010-02-19


In [11]:
relev_insom_events_by_pt = relev_insom_events.groupby(by='patid')['eventdate'].count().reset_index()
relev_insom_events_by_pt.columns=['patid','insomnia_event_count']

In [46]:
merged_table = pd.merge(pt_features,relev_insom_events_by_pt, on='patid',how='outer')
merged_table['insomnia_event_count'].fillna(0,inplace=True)
merged_table['insomnia_event_count'] = merged_table['insomnia_event_count'].astype(int)

In [47]:
merged_table

,patid,pracid,gender,birthyear,index_date,original_matchid,isCase,insomnia_event_count
0,4923328,328,2,1916,2003-07-01,4355328,True,0
1,1959216,216,1,1933,2011-02-25,6290216,True,0
2,9271420,420,1,1917,2003-06-20,2770420,True,0
3,548429,429,2,1925,2005-05-11,5473429,True,0
4,14185081,81,2,1917,2005-02-28,5951081,True,0
5,7329438,438,2,1941,2008-08-04,17742438,True,0
6,6327373,373,1,1941,2009-03-26,5262373,True,0
7,5811024,24,1,1930,2010-12-14,6394024,True,0
8,6905371,371,1,1926,2006-09-19,15830371,True,0
9,7741629,629,1,1936,2004-06-10,9053629,True,0


In [50]:
merged_table[merged_table['isCase']==True]['insomnia_event_count'].mean()

0.08558000965109204

In [51]:
merged_table[merged_table['isCase']==False]['insomnia_event_count'].mean()

0.068795501751883009

In [57]:
pt_features[pt_features['patid']==3176659]

,patid,pracid,gender,birthyear,index_date,original_matchid,isCase
1518,3176659,659,2,1926,2011-10-18,7877659,True
